In [2]:
import os

In [1]:
%pwd

'c:\\Users\\Sadeeq\\Desktop\\Krish Naik\\End to End Project\\ML_Pipeline\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Sadeeq\\Desktop\\Krish Naik\\End to End Project\\ML_Pipeline'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [14]:
from src.mlproject.constants import *
from src.mlproject.utils import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config
        

In [26]:
import os
import urllib.request as request
from src.mlproject import logger
import zipfile

In [29]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        # Logic to download data from self.config.source_URL
        if not os.path.exists(self.config.local_data_file):
            filename, headers =  request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"File downloaded: {filename} with headers: {headers}")
        else:
            logger.info(f"File already exists: {self.config.local_data_file}")
    
    def extract_zip_file(self):
        # Logic to unzip data to self.config.unzip_dir
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted files to: {unzip_path}")

In [30]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-06 19:50:42,390: INFO: __init__ - YAML file read successfully from config\config.yaml]
[2025-07-06 19:50:42,392: INFO: __init__ - YAML file read successfully from params.yaml]
[2025-07-06 19:50:42,394: INFO: __init__ - YAML file read successfully from schema.yaml]
[2025-07-06 19:50:42,396: INFO: __init__ - Directory created at artifacts]
[2025-07-06 19:50:42,396: INFO: __init__ - Directory created at artifacts/data_ingestion]
[2025-07-06 19:50:42,397: INFO: 592378645 - File already exists: artifacts/data_ingestion/data.zip]
[2025-07-06 19:50:42,399: INFO: 592378645 - Extracted files to: artifacts/data_ingestion]
